## Prepare Data para o fine tuning do GPT

O código realiza a preparação de prompts a partir de um arquivo JSON de entrada, limpando o texto e formatando-o de uma maneira específica. Aqui está uma explicação detalhada:

1. **Importação de Módulos**:
   - `json`: Para manipulação de dados JSON.
   - `re`: Para operações com expressões regulares.
   - `html`: Para manipulação de entidades HTML.

2. **Função `clean_text`**:
   - Remove espaços extras e caracteres indesejados do texto.
   - Converte entidades HTML para caracteres normais.
   - Substitui caracteres específicos como NBSP e SHY.

3. **Função `prepare_prompts`**:
   - Lê o arquivo JSON de entrada linha por linha.
   - Inicializa uma lista para armazenar os dados processados.
   - Itera sobre cada linha, convertendo-a em um dicionário Python.
   - Limpa e verifica se o título e o conteúdo não estão vazios.
   - Formata o prompt com o título e o conteúdo e adiciona à lista de dados processados.
   - Salva os dados processados em um arquivo JSON de saída.

4. **Execução da Função**:
   - Define os caminhos dos arquivos de entrada e saída.
   - Chama a função `prepare_prompts` com os caminhos definidos.
   - Imprime uma mensagem indicando que os prompts foram salvos.

In [1]:
import json
import re
import html
import unicodedata

In [2]:
# Define uma função para limpar o texto
def clean_text(text):
    # Remove caracteres indesejados e espaços extras
    text = re.sub(r'\s+', ' ', text)  # Substitui múltiplos espaços por um único espaço
    text = text.strip()  # Remove espaços extras no início e no final do texto
    text = html.unescape(text)  # Converte entidades HTML para caracteres normais
    # Ajusta os caracteres Unicode
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')
    text = text.encode('unicode_escape').decode('unicode_escape')
    return text

# Define uma função para preparar os prompts
def prepare_prompts(file_path, output_path):
    # Abre o arquivo JSON de entrada para leitura
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()  # Lê todas as linhas do arquivo

    processed_data = []  # Inicializa uma lista para armazenar os dados processados

    # Itera sobre cada linha do arquivo
    for line in lines[:50000]:
        item = json.loads(line)  # Converte a linha de texto JSON em um dicionário Python
        title = clean_text(item.get('title', '')).strip()
        content = clean_text(item.get('content', '')).strip()

        # Verifica se o título e o conteúdo não estão vazios
        if title and content and title != content:
            # Formata o prompt com o título e o conteúdo
            prompt = f"DESCRIBE THE TITLE BASED ON THE CONTENT\n[|Title|] {title}[|eTitle|]\n\n[|Content|]{content}[|eContent|]"
            processed_data.append({"input": prompt})  # Adiciona o prompt formatado à lista de dados processados

    # Abre o arquivo JSON de saída para escrita
    with open(output_path, 'w', encoding='utf-8') as output_file:
        # Salva os dados processados no arquivo JSON de saída
        json.dump(processed_data, output_file, ensure_ascii=False, indent=4)

# Caminho para o arquivo JSON de entrada e o arquivo JSON de saída
input_file_path = './drive/MyDrive/FIAP/data/trn.json'
output_file_path = './drive/MyDrive/FIAP/processed/processed_prompts.json'

# Chama a função para preparar os prompts
prepare_prompts(input_file_path, output_file_path)

# Imprime uma mensagem indicando que os prompts foram salvos
print(f"Prompts preparados foram salvos em '{output_file_path}'.")

Prompts preparados foram salvos em './drive/MyDrive/FIAP/processed/processed_prompts.json'.


## Tokenização e Codificação dos Dados

Vamos utilizar unsloth para tokenizar o texto e codificar os dados para o treinamento do modelo GPT.

#### Vamos precisar utilizar a execução do colab por GPU



1.   Habilitar a GPU:
  - No Colab, vá em "Ambiente de execução" -> "Alterar tipo de ambiente de execução".
  - Na seção "Acelerador de hardware", selecione "GPU" no menu suspenso.
Clique em "Salvar".


2.   Reiniciar o Ambiente de Execução:
  - Após alterar o tipo de ambiente de execução, é importante reiniciar o ambiente para que as mudanças sejam aplicadas. Você pode fazer isso indo em "Ambiente de execução" -> "Reiniciar ambiente de execução".









In [3]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-ksruff7s/unsloth_e6a6c68427594e3ea82e0c30cd68b365
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-ksruff7s/unsloth_e6a6c68427594e3ea82e0c30cd68b365
  Resolved https://github.com/unslothai/unsloth.git to commit 228b3cf46ec4401b81194267ed0091eb62a56c6b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
DATA_PATH = "./drive/MyDrive/FIAP/processed/processed_prompts.json"
OUTPUT_PATH_DATASET = "./drive/MyDrive/FIAP/processed/processed_prompts_dataset.json"
max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

In [5]:
def format_dataset_into_model_input(data):
    def separate_text(full_text):
        title_start = full_text.find("[|Title|]") + len("[|Title|]")
        title_end = full_text.find("[|eTitle|]")
        content_start = full_text.find("[|Content|]") + len("[|Content|]")
        content_end = full_text.find("[|eContent|]")

        instruction = full_text.split('\n')[0]
        input_text = full_text[title_start:title_end].strip()
        response = full_text[content_start:content_end].strip()

        return instruction, input_text, response

    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Processando o dataset
    for item in data:  # Iterar diretamente sobre a lista de dicionários
        instruction, input_text, response = separate_text(item['input'])
        instructions.append(instruction)
        inputs.append(input_text)
        outputs.append(response)

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON
    with open(OUTPUT_PATH_DATASET, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

    print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")

#### Vamos carregar nossos dados de prompt

In [6]:
with open(DATA_PATH, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [7]:
from unsloth import FastLanguageModel, is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [8]:
format_dataset_into_model_input(data)

Dataset salvo em ./drive/MyDrive/FIAP/processed/processed_prompts_dataset.json


In [9]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.11.9: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

#### Agora vamos carregar nosso modelo

Aqui vamos carregar o modelo que baixamos

In [18]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Already have LoRA adapters! We shall skip this step.


Este trecho de código está aplicando uma técnica chamada PEFT (Parameter-Efficient Fine-Tuning) ao modelo de linguagem `model` previamente carregado. O objetivo do PEFT é ajustar o modelo para uma tarefa específica de forma eficiente, modificando apenas uma pequena parte dos parâmetros do modelo original. Isso economiza tempo e recursos computacionais em comparação com o ajuste fino tradicional, que treina todos os parâmetros.

Vamos entender os parâmetros da função `FastLanguageModel.get_peft_model`:

1. model: O modelo de linguagem base que será ajustado.

2. r = 16: Define o rank da matriz LoRA (Low-Rank Adaptation), que é a técnica principal usada pelo PEFT. Um rank menor significa menos parâmetros a serem ajustados, tornando o processo mais eficiente.

3. target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",]: Especifica os módulos do modelo base onde as matrizes LoRA serão aplicadas. Esses módulos geralmente são partes importantes da arquitetura do modelo, como camadas de atenção.

4. lora_alpha = 16: Um fator de escala que controla a influência das matrizes LoRA no processo de ajuste fino.

5. lora_dropout = 0: A taxa de dropout aplicada às matrizes LoRA. Dropout é uma técnica de regularização que ajuda a evitar o overfitting.

6. bias = "none": Indica que nenhum bias (termo de polarização) será adicionado às matrizes LoRA.

7. use_gradient_checkpointing = "unsloth": Habilita uma técnica chamada gradient checkpointing, que reduz o consumo de memória durante o treinamento. O valor "unsloth" provavelmente se refere a uma implementação específica dessa técnica pela biblioteca Unsloth.

8. random_state = 3407: Define a semente para o gerador de números aleatórios, garantindo a reprodutibilidade dos resultados.

9. use_rslora = False: Desabilita o uso de uma variante do LoRA chamada "Rotary Sparse LoRA" (RSLoRA).

10. loftq_config = None: Desabilita o uso de outra técnica de quantização chamada "Low-bit Optimization of Quantized Transformers" (LOFTQ).

**Em resumo, este código está preparando o modelo de linguagem para um ajuste fino eficiente utilizando a técnica PEFT com LoRA. Ele configura os parâmetros da adaptação, especificando onde as modificações serão aplicadas e como elas influenciarão o treinamento.** O resultado final é um modelo ajustado para a tarefa desejada, com um menor custo computacional em comparação ao ajuste fino tradicional.

In [19]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset


dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [20]:
def test_model_alpaca(instruction, input_text=""):
    global model
    # Preparar o modelo para inferência
    model = FastLanguageModel.for_inference(model)

    # Formatar o prompt no formato Alpaca
    prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n"

    # Tokenizar o prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(device="cuda")

    # Gerar a saída pelo modelo
    outputs = model.generate(**inputs)

    # Decodificar a saída
    predicted_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extrair a resposta após "### Response:"
    predicted_response = predicted_response.split("### Response:")[-1].strip()

    return predicted_response

In [21]:
title_question_example = "The GOOD LUCK PENCIL"  # Exemplo de título como pergunta

predicted_content = test_model_alpaca(title_question_example)  # Chama a função com o título
print(f"Título/Pergunta: {title_question_example}")
print(f"Conteúdo:\n{predicted_content}")  # Imprime o conteúdo

Título/Pergunta: The GOOD LUCK PENCIL
Conteúdo:
The GOOD LUCK PENCIL

### Hint:
The GOOD LUCK PENCIL

### Solution:
The GOOD LUCK PENCIL


### Criando o treinamento

In [22]:
from trl import SFTTrainer
from transformers import TrainingArguments

In [23]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to="none",
    ),
)

Map (num_proc=2):   0%|          | 0/38395 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [24]:
import torch

torch.cuda.empty_cache()

import gc
gc.collect()

trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 38,395 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 8
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.661300
2,2.684400
3,2.560900
4,2.778100
5,2.821200
6,2.454100
7,2.301400
8,2.253100
9,2.049500
10,2.302500


In [25]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format("DESCRIBE THE TITLE BASED ON THE CONTENT.",
        "The GOOD LUCK PENCIL",
        "",
  )
], return_tensors="pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)


['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nDESCRIBE THE TITLE BASED ON THE CONTENT.\n\n### Input:\nThe GOOD LUCK PENCIL\n\n### Response:\nGr. 2-4--This is a story of a pencil that has been used by a number of children and is now passed on to a new owner, a little girl named Mary. The pencil has been passed on by a number of children, and each one has added his or her own words to the story']

Resultado obtido:


Gr. 2-4--Esta é a história de um lápis que foi usado por várias crianças e agora foi passado para uma nova dona, uma garotinha chamada Mary. O lápis foi repassado por diversas crianças e cada uma acrescentou suas próprias palavras à história

História do dataset (Inglês):

Mary Ann can't do anything wrongespecially after finding a good-luck pencil. She works all the math problems right and draws pictures that everyone admires. When her teacher assigns a homework composition, Mary Ann must find something interesting to write about her ordinary family, so she lets the good-luck pencil go to work. It writes that her mother is a world-famous ballerina, her father is an astronaut and Mary Ann a world champion piano player planning to climb Mt. Olympus in the summer. But then the composition becomes real. After practicing Mozart and Bach over and over, then doing hours of exhausting exercises, Mary Ann longs for a regular life again. She pulls out a different pencil and writes a less exciting but accurate composition, dispelling the unwanted illusion. The illustrations are lighthearted and will draw young readers into repeated readings.Copyright 1986 Reed Business Information, Inc.

(Português):

Mary Ann não pode fazer nada de errado, especialmente depois de encontrar um lápis da sorte. Ela resolve todos os problemas de matemática corretamente e faz desenhos que todos admiram. Quando sua professora atribui uma redação de lição de casa, Mary Ann deve encontrar algo interessante para escrever sobre seu cotidiano a família, então ela deixa o lápis da sorte trabalhar. Ele escreve que sua mãe é uma bailarina mundialmente famosa, seu pai é um astronauta e Mary Ann uma pianista campeã mundial que planeja escalar o Monte. então a composição se torna real Depois de praticar Mozart e Bach repetidamente e depois fazer horas de exercícios exaustivos, Mary Ann anseia por uma vida normal novamente. Ela pega um lápis diferente e escreve uma composição menos emocionante, mas precisa, dissipando a ilusão indesejada. alegre e atrairá jovens leitores para leituras repetidas. Copyright 1986 Reed Business Information, Inc.
